In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_diagonal_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,1218.64,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00
3,-0.00,290.55,-0.00,0.00,0.00,0.00,0.00,-0.00
4,0.00,-0.00,60.99,-0.00,-0.00,0.00,-0.00,0.00
5,-0.00,0.00,-0.00,12.19,0.00,0.00,0.00,-0.00
6,-0.00,0.00,-0.00,0.00,2.42,-0.00,0.00,0.00
7,-0.00,0.00,0.00,0.00,-0.00,0.52,-0.00,-0.00
8,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.15,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.05


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,0.00000,-0.00003,-0.00004,-0.00002,-0.00005,0.00001
3,-0.00001,1.00000,-0.00000,0.00003,0.00001,0.00004,0.00000,-0.00003
4,0.00000,-0.00000,1.00000,-0.00004,-0.00002,0.00006,-0.00002,0.00003
5,-0.00003,0.00003,-0.00004,1.00000,0.00002,0.00000,0.00001,-0.00004
6,-0.00004,0.00001,-0.00002,0.00002,1.00000,-0.00001,0.00003,0.00006
7,-0.00002,0.00004,0.00006,0.00000,-0.00001,1.00000,-0.00002,-0.00002
8,-0.00005,0.00000,-0.00002,0.00001,0.00003,-0.00002,1.00000,0.00002
9,0.00001,-0.00003,0.00003,-0.00004,0.00006,-0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.008700859589944718

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.85872461e+05 1.03379393e+04 4.43047989e+02 1.69853742e+01
 6.31877006e-01 2.62917066e-02 1.84021105e-03 7.28105833e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999324,-0.036030,-0.007092,-0.001416,-0.000283,-0.000061,-0.000018,-0.000005
1,0.035735,0.998658,-0.036804,-0.006982,-0.001387,-0.000296,-0.000089,-0.000028
2,0.008328,0.036193,0.998402,-0.041821,-0.007938,-0.001693,-0.000507,-0.000154
3,0.001990,0.008341,0.041040,0.997708,-0.051944,-0.010616,-0.003154,-0.000969
4,0.000492,0.002056,0.009816,0.050544,0.995619,-0.074855,-0.021035,-0.006404
5,0.000135,0.000561,0.002676,0.013439,0.069646,0.983910,-0.158305,-0.042690
6,0.000056,0.000232,0.001110,0.005550,0.027959,0.136582,0.943086,-0.301862
7,0.000037,0.000156,0.000740,0.003710,0.018625,0.086880,0.291677,0.952373


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006755037644725626,
 0.0013420434813803395,
 0.0015984846413196196,
 0.002292405767911898,
 0.004380722874743226,
 0.01609025911953632,
 0.05691408663721853,
 0.04762665500668306]